In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import io

import numpy as np
import pymongo

from bokeh.io import output_notebook, show
from bokeh.palettes import all_palettes
from bokeh.layouts import gridplot

from IPython import display

from climatools.cliradlw.dataset import load_output_file
import climatools.html.html as climahtml
from climatools.plot.plot import *

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
def make_query(param=None):
    '''
    Returns the MongoDB query for a lblnew 
    run's document.
    
    Parameters
    ----------
    param: dict
    lblnew-bestfit input parameters.
    '''
    return {'param.' + name: value for name, value in param.items()}   

In [5]:
def print_input_param():
    '''
    Print to screen the input parameters 
    of the lblnew run.
    '''
    df = pd.DataFrame(columns=['lblnew'])
    for name, value in sorted(PARAM.items()):
        df.loc[name, 'lblnew'] = value
    
    df.index.name = 'Input parameter'
    df.columns.name = 'Run'
    
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
        
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band') 
            
    return ds['coolrg']



def pltdata_cool(g=None):
    '''
    Define the plotting data for cooling rate
    profile plots.  Add/remove entries in
    the `data` list.
    
    Parameters
    ----------
    g: int
        g-interval to be plotted.  If None,
        the sum over all g-intervals will be
        plotted.
    data: dict
        Plot data, including colour, line attributes, etc.
    '''
    colors = all_palettes['Set1'][4]
    
    if g == None:
        ds_cool_crd = DS_COOL_CRD.sum('g')
        ds_cool_wgt = DS_COOL_WGT.sum('g')
    else:
        ds_cool_crd = DS_COOL_CRD.sel(g=g)
        ds_cool_wgt = DS_COOL_WGT.sel(g=g)
    
    data = [
        {'label': 'CRD',
         'srs': fmt_cool(ds_cool_crd),
         'line_dash': 'solid', 'line_width':1.5,
         'marker': 'circle', 'mark_size': 5,
         'color': colors[2], 'alpha': 1},
        {'label': 'WGT igg=1',
         'srs': fmt_cool(ds_cool_wgt.sel(igg=1)),
         'line_dash': 'solid', 'line_width': 3,
         'marker': 'square', 'marker_size': 5,
         'color': colors[3], 'alpha': .6},
        {'label': 'WGT igg=10',
         'srs': fmt_cool(ds_cool_wgt.sel(igg=10)),
         'line_dash': 'solid', 'line_width': 2,
         'marker': 'triangle', 'marker_size': 2,
         'color': colors[1], 'alpha': .3}]
    return data



def plt_cool(pltdata=None):
    '''
    Plot cooling rate profiles on linear
    and log pressure axes.
    
    Parameters
    ----------
    pltdata: dict
        Plot data, including colour, line attributes, etc.
    everything: bokeh.models.layouts.Column
        Cooling rate profile plot.
    '''
    plin = plt_vert_profile_bokeh(pltdata=pltdata, 
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200)) 
    plog.plot_width, plog.plot_height = 400, 500
    
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
    

def script_plt_cool():
    '''
    Script to plot the cooling rate profiles
    in each g-interval and the total 
    cooling rate profile over all g-intervals.
    '''
    ng = sum(PARAM['ng_refs'])
    for g in np.arange(1, ng + 1):
        data = pltdata_cool(g=g)
        p = plt_cool(pltdata=data)
        show(p)
        display.display(
            display.Markdown(f'*FIGURE.* Cooling rates. g={g}'))
    
    data = pltdata_cool()
    p = plt_cool(pltdata=data)
    show(p)
    display.display(
        display.Markdown(f'*FIGURE.* Cooling rates. Summed over g.'))
    


In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    #if 'igg' in ds.dims:
    #    ds = ds.squeeze('igg').drop('igg')
        
    if 'igg' in ds:
        if ds['igg'].shape == (1,):
            ds = ds.squeeze('igg').drop('igg')
        elif ds['igg'].shape == ():
            ds = ds.drop('igg')
            
    if 'g' in ds:
        if ds['g'].shape == (1,):
            ds = ds.squeeze('g').drop('g')
        elif ds['g'].shape == ():
            ds = ds.drop('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux():
    '''
    Define the flux data to be tabulated for
    the g-intervals.  Add/remove entries in the
    `tbs` list.
    
    Parameters
    ----------
    data: dict
        Flux data, including how each set is to be
        labeled in the output table.
    '''
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=1',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=[1]), ilevels=ilevels)},
           {'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=[10]), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data



def tbdata_flux_sumg():
    '''
    Define the total flux data, over all g-intervals,
    to be tabulated.
    
    Parameters
    ----------
    data: dict
        Flux data, including how each set is to e
        labeled in the output table.
    '''
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=1',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=[1]).sum('g'), 
                           ilevels=ilevels)},
           {'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=[10]).sum('g'), 
                           ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    '''
    Display flux table for each set, and display
    the difference between each set and the set
    that is specified to be the benchmark.
    
    Parameters
    ----------
    tbdata: dict
        Flux data, including how each set it to be
        labeled in the output table.
    '''
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        display.display(display.Markdown(
            f"*TABLE.* Fluxes. {d['label']}"))
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        display.display(display.Markdown(
            f"*TABLE.* Fluxes. ({d['label']}) - ({d0['label']})"))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        

        
def script_tb_flux():
    '''
    Script to display the flux tables in 
    each g-interval, and the flux table
    for the total flux, over all g-intervals.
    '''
    tb_flux(tbdata=tbdata_flux())
    tb_flux(tbdata=tbdata_flux_sumg())


In [8]:
def script_gsummary():
    '''
    Script to display how the g-intervals
    are defined according the hyperparameters
    chosen for this lblnew-bestfit run.
    '''
    ng_refs = PARAM['ng_refs']
    p_refs = list(zip(*PARAM['ref_pts']))[0]
    ng = sum(ng_refs)
    df_abscom = pd.read_csv(io.StringIO(DOC['abscom']), 
                            sep=r'\s+', index_col=[0])
    df_dgdgs = pd.read_csv(io.StringIO(DOC['dgdgs']), 
                            sep=r'\s+', index_col=[0])
    
    # absth(g + 1) / absth(g)
    df_abscom_copy = df_abscom.copy()
    df_abscom_copy.index = df_abscom.index + 1
    df_absratio = df_abscom / df_abscom_copy
    df_absratio.columns = ['k(ig) / k(ig - 1)']
    df_absratio = df_absratio.loc[: ng]    
    
    df_dgdgs_cumsum = df_dgdgs.cumsum(axis=0)
    df_dgdgs_cumsum.columns = ['dg cumsum', 'dgs cumsum']
    
    data_pref = np.concatenate([p_ref * np.ones((ng_ref,)) 
                                for ng_ref, p_ref 
                                in zip(ng_refs, p_refs)], axis=0)
    df_pref = pd.DataFrame(data_pref, 
                           index=df_dgdgs.index, columns=['pref'])
    
    df_all = pd.concat([df_abscom, df_absratio,
                        df_dgdgs, df_dgdgs_cumsum,
                        df_pref], axis=1) 
    
    display.display(df_all)
    display.display(display.Markdown(
        (f"TABLE. g-interval summary.  "
         f"*abscom* is the lower bound of the g-interval.")))
    


In [9]:
def script_fluxcool_to_excel():
    fname = 'lblnew-bestfit_fluxcool.xlsx'
    writer = pd.ExcelWriter(fname)    

    gs = DS_FLUX_CRD.coords['g']
    for g in gs:
        ds = DS_FLUX_CRD
        df0 = ds.sel(g=g).to_dataframe()
        df0.drop('g', axis=1, inplace=True)
        df0.set_index('level', append=True, inplace=True)

        ds = DS_FLUX_WGT
        iggs = [1, 10]
        dfwgts = [ds.sel(g=g, igg=igg).to_dataframe() for igg in iggs]
        [df.drop(['g', 'igg'], axis=1, inplace=True) for df in dfwgts]
        [df.set_index('level', append=True, inplace=True) for df in dfwgts]
        dfdifs = [df - df0 for df in dfwgts]
        dfwgts = [pd.concat([dfwgt, dfdif], axis=1, keys=['WGT', 'WGT - CRD'], names=['model']) 
                  for dfwgt, dfdif in zip(dfwgts, dfdifs)]
        dfwgt = pd.concat(dfwgts, axis=1, keys=iggs, names=['igg'])
        
        df0.columns = pd.MultiIndex.from_tuples([('N/A', 'CRD', v) for v in df0.columns], 
                                                names=dfwgt.columns.names)
        df = pd.concat([df0, dfwgt], axis=1)
        df.to_excel(writer, f'flux g={int(g)}')
        
    ds = DS_FLUX_CRD
    df0 = ds.sum('g').to_dataframe()
    df0.set_index('level', append=True, inplace=True)
    ds = DS_FLUX_WGT
    iggs = [1, 10]
    dfwgts = [ds.sum('g').sel(igg=igg).to_dataframe() for igg in iggs]
    [df.drop(['igg'], axis=1, inplace=True) for df in dfwgts]
    [df.set_index('level', append=True, inplace=True) for df in dfwgts]
    dfdifs = [df - df0 for df in dfwgts]
    dfwgts = [pd.concat([dfwgt, dfdif], axis=1, keys=['WGT', 'WGT - CRD'], names=['model'])
              for dfwgt, dfdif in zip(dfwgts, dfdifs)]
    dfwgt = pd.concat(dfwgts, axis=1, keys=iggs, names=['igg'])
    df0.columns = pd.MultiIndex.from_tuples([('N/A', 'CRD', v) for v in df0.columns],
                                           names=dfwgt.columns.names)
    df = pd.concat([df0, dfwgt], axis=1)
    df.to_excel(writer, f'flux sum(g)')
    
    gs = DS_COOL_CRD.coords['g']
    for g in gs:
        ds = DS_COOL_CRD
        df0 = ds.sel(g=g).to_dataframe()
        df0.drop('g', axis=1, inplace=True)
        df0.set_index('layer', append=True, inplace=True)

        ds = DS_COOL_WGT
        iggs = [1, 10]
        dfwgts = [ds.sel(g=g, igg=igg).to_dataframe() for igg in iggs]
        [df.drop(['g', 'igg'], axis=1, inplace=True) for df in dfwgts]
        [df.set_index('layer', append=True, inplace=True) for df in dfwgts]
        dfdifs = [df - df0 for df in dfwgts]
        dfwgts = [pd.concat([dfwgt, dfdif], axis=1, keys=['WGT', 'WGT - CRD'], names=['model']) 
                  for dfwgt, dfdif in zip(dfwgts, dfdifs)]
        dfwgt = pd.concat(dfwgts, axis=1, keys=iggs, names=['igg'])
        
        df0.columns = pd.MultiIndex.from_tuples([('N/A', 'CRD', df0.columns[0])], names=dfwgt.columns.names)
        df = pd.concat([df0, dfwgt], axis=1)
        df.to_excel(writer, f'cool g={int(g)}')

    ds = DS_COOL_CRD
    df0 = ds.sum('g').to_dataframe()
    df0.set_index('layer', append=True, inplace=True)
    ds = DS_COOL_WGT
    iggs = [1, 10]
    dfwgts = [ds.sum('g').sel(igg=igg).to_dataframe() for igg in iggs]
    [df.drop(['igg'], axis=1, inplace=True) for df in dfwgts]
    [df.set_index('layer', append=True, inplace=True) for df in dfwgts]
    dfdifs = [df - df0 for df in dfwgts]
    dfwgts = [pd.concat([dfwgt, dfdif], axis=1, keys=['WGT', 'WGT - CRD'], names=['model'])
              for dfwgt, dfdif in zip(dfwgts, dfdifs)]
    dfwgt = pd.concat(dfwgts, axis=1, keys=iggs, names=['igg'])
    df0.columns = pd.MultiIndex.from_tuples([('N/A', 'CRD', df0.columns[0])], names=dfwgt.columns.names)
    df = pd.concat([df0, dfwgt], axis=1)
    df.to_excel(writer, f'cool sum(g)')
        
    writer.save()
    html_download = f'Download: <a href="./{fname}">{fname}</a>'
    display.display(display.HTML(html_download))    

In [10]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

In [11]:
def script():
    title = ('### lblnew-bestfit' 
             if 'ng_refs' in PARAM else '### lblnew-overlap')
    s_input = 'Input Parameters'
    s_plt_cool = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_gsummary = 'g-interval Summary'
    s_download = 'Download Data'
    
    # title
    show_markdown(title)
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cool))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_gsummary))
    show_html(climahtml.getHTML_hrefanchor(s_download))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cool))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cool))
    script_plt_cool()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # g-interval summary
    show_html(climahtml.getHTML_idanchor(s_gsummary))
    show_markdown(climahtml.getMarkdown_sectitle(s_gsummary))
    script_gsummary()
    
    # Download cooling rate and fluxes
    show_html(climahtml.getHTML_idanchor(s_download))
    show_markdown(climahtml.getMarkdown_sectitle(s_download))
    script_fluxcool_to_excel()

In [12]:
%run param.py

client = pymongo.MongoClient('localhost', 27017, connect=False)
db = client.lblnew


DOC = db.bestfit_lw.find_one(make_query(param=PARAM))
DS_FLUX_CRD = load_output_file(io.StringIO(DOC['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(DOC['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(DOC['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(DOC['output_wcoolrg']))

In [13]:
script()

### lblnew-bestfit

### Table of Contents

# Input Parameters

Run,lblnew
Input parameter,
atmpro,trp
band,3c
commitnumber,a22ab94
conc,0.0004
dv,0.001
klin,6.37556e-24
molecule,co2
ng_adju,"[0, 0, 0]"
ng_refs,"[3, 2, 4]"


# Cooling Rate Profiles

...

*TABLE.* Fluxes. (WGT igg=10) - (CRD)

# g-interval Summary

,abscom,k(ig) / k(ig - 1),dg,dgs,dg cumsum,dgs cumsum,pref
g,,,,,,,
1,2.310867e-19,NaN,0.000850,0.000901,0.000850,0.000901,1.0
2,2.133976e-20,0.092345,0.001075,0.001127,0.001925,0.002028,1.0
3,1.970626e-21,0.092345,0.006450,0.006712,0.008375,0.008740,1.0
4,1.774707e-21,0.900580,0.011150,0.011783,0.019525,0.020522,10.0
5,1.647913e-22,0.092856,0.038525,0.040010,0.058050,0.060533,10.0
6,5.838237e-22,3.542806,0.068675,0.071889,0.126725,0.132421,500.0
7,6.101170e-23,0.104504,0.290038,0.301040,0.416763,0.433461,500.0
8,6.375943e-24,0.104504,0.213775,0.212014,0.630537,0.645476,500.0
9,2.959058e-25,0.046410,0.369463,0.354524,1.000000,1.000000,500.0


TABLE. g-interval summary.  *abscom* is the lower bound of the g-interval.

# Download Data

In [14]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')